# Buyout Decision Notebook

## Authors:
- Farinaz Motlagh, Stony Brook University
- Sara Hamideh, Stony Brook University

In [ ]:
import geopandas as gpd
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import pandas as pd 
from osgeo import gdal
import rasterio
from rasterio.features import rasterize
from rasterio.mask import mask
import random
from shapely.geometry import Point
from shapely.wkt import loads
from matplotlib.table import Table
from IPython.display import display, Markdown
from geopy.distance import great_circle
from heapq import nsmallest
from geopy.distance import geodesic
from itertools import combinations
import seaborn as sns
import contextily as ctx

import warnings
warnings.filterwarnings('ignore')

from pyincore import IncoreClient, DataService, Dataset, FragilityService, MappingSet
from pyincore.utils.dataprocessutil import DataProcessUtil
# importing pyIncone analyses:
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.nonstructbuildingdamage import NonStructBuildingDamage
from pyincore.analyses.combinedwindwavesurgebuildingdamage import CombinedWindWaveSurgeBuildingDamage
from pyincore.analyses.populationdislocation import PopulationDislocation
from pyincore.analyses.buyoutdecision import BuyoutDecision

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
# Connect to IN-CORE Services
client = IncoreClient()
client.clear_cache()
data_service = DataService(client)

## Data Preparation

In [ ]:
# Loading Galveston building inventory
bldg_dataset_id = "63ff6b135c35c0353d5ed3ac"
buildings = Dataset.from_data_service(bldg_dataset_id, data_service)
bld_invtry = buildings.get_dataframe_from_shapefile()
bld_invtry.head()
#bld_invtry.shape

In [ ]:
# Removing any properties that have NaN values and values less than $1000 in column "appr_bldg"
# (the threshold is based on Galveston County's tax assessor data for mobile homes)
bld_invtry = bld_invtry[bld_invtry['appr_bldg'].notna() & (bld_invtry['appr_bldg'] >= 1000)]
#bld_invtry.shape

## Max damage state from Combined Building Damage Analyses

The decision framework selects candidate properties for buyout projects that (1) have been completely damaged due to a past hurricane scenario and (2) are projected to be completely damaged due to a 100-year hurricane event in the future. To check for the first requirement, we will use IN-CORE's Hurricane Ike's hindcast of combined building damage model and will include properties that have been completely damaged during this event. Next, we will use IN-CORE's projected building damage model output from a 100-year event in the future to check for the second requirement. The building damage model is developed from fragility analysis to estimate the probability of four damage states based on the hazard type and attributes of the building.


In [ ]:
def calculate_combined_building_damage(hazard_type, hazard_id, bldg_dataset_id, wind_mapping_id,
                                      surge_wave_mapping_id, flood_mapping_id,result_name):

    # Wind building damage
    fragility_service = FragilityService(client)
    wind_mapping_set = MappingSet(fragility_service.get_mapping(wind_mapping_id))
    w_bldg_dmg = BuildingDamage(client)
    w_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
    w_bldg_dmg.set_input_dataset('dfr3_mapping_set', wind_mapping_set)
    w_bldg_dmg.set_parameter("result_name", "Galveston-wind-dmg")
    w_bldg_dmg.set_parameter("hazard_type", hazard_type)
    w_bldg_dmg.set_parameter("hazard_id", hazard_id)
    w_bldg_dmg.set_parameter("num_cpu", 8)
    w_bldg_dmg.run_analysis()

    # surge-wave building damage
    sw_bldg_dmg = BuildingDamage(client)
    surge_wave_mapping_set = MappingSet(fragility_service.get_mapping(surge_wave_mapping_id))
    sw_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
    sw_bldg_dmg.set_input_dataset('dfr3_mapping_set', surge_wave_mapping_set)
    sw_bldg_dmg.set_parameter("result_name", "Galveston-sw-dmg")
    sw_bldg_dmg.set_parameter("hazard_type", hazard_type)
    sw_bldg_dmg.set_parameter("hazard_id", hazard_id)
    sw_bldg_dmg.set_parameter("num_cpu", 8)
    sw_bldg_dmg.run_analysis()

    # Flood mapping damage
    flood_mapping_set = MappingSet(fragility_service.get_mapping(flood_mapping_id))
    f_bldg_dmg = NonStructBuildingDamage(client)
    f_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
    f_bldg_dmg.set_input_dataset('dfr3_mapping_set', flood_mapping_set)
    f_bldg_dmg.set_parameter("result_name", "Galveston-flood-dmg")
    f_bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
    f_bldg_dmg.set_parameter("hazard_type", hazard_type)
    f_bldg_dmg.set_parameter("hazard_id", hazard_id)
    f_bldg_dmg.set_parameter("num_cpu", 8)
    f_bldg_dmg.run_analysis()

    # Running combined damage analysis
    surge_wave_damage = sw_bldg_dmg.get_output_dataset("ds_result")
    wind_damage = w_bldg_dmg.get_output_dataset("ds_result")
    flood_damage = f_bldg_dmg.get_output_dataset("result")
    
    combined_bldg_dmg = CombinedWindWaveSurgeBuildingDamage(client)
    combined_bldg_dmg.set_input_dataset("surge_wave_damage", surge_wave_damage)
    combined_bldg_dmg.set_input_dataset("wind_damage", wind_damage)
    combined_bldg_dmg.set_input_dataset("flood_damage", flood_damage)
    combined_bldg_dmg.set_parameter("result_name", result_name)
    combined_bldg_dmg.run_analysis()

    # Returning combined damage result 
    combined_dmg = combined_bldg_dmg.get_output_dataset("ds_result")
    max_dmg = combined_dmg.get_dataframe_from_csv()
    
    return combined_dmg

## Get max damage states

In [ ]:
# Hurricane Ike Building combined damage

wind_mapping_id = "62fef3a6cef2881193f2261d"
surge_wave_mapping_id = "6303e51bd76c6d0e1f6be080"
flood_mapping_id = "62fefd688a30d30dac57bbd7"


hazard_type = "hurricane"
hazard_id = "5fa5a228b6429615aeea4410"
bldg_dataset_id = "63ff6b135c35c0353d5ed3ac"
result_name = "Galveston-Ike-combined-dmg"

combined_dmg_ike = calculate_combined_building_damage(hazard_type, hazard_id, bldg_dataset_id, wind_mapping_id,
                                      surge_wave_mapping_id, flood_mapping_id,result_name)

In [ ]:
# Hurricane Ike 100 year Building combined damage
wind_mapping_id = "62fef3a6cef2881193f2261d"
surge_wave_mapping_id = "6303e51bd76c6d0e1f6be080"
flood_mapping_id = "62fefd688a30d30dac57bbd7"


hazard_type = "hurricane"
hazard_id = "5fa5a9497e5cdf51ebf1add2"
bldg_dataset_id = "63ff6b135c35c0353d5ed3ac"
result_name = "Galveston-Ike-100yr-combined-dmg"

combined_dmg_ike_100yrs  = calculate_combined_building_damage(hazard_type, hazard_id, bldg_dataset_id, wind_mapping_id,
                                      surge_wave_mapping_id, flood_mapping_id,result_name)

### Population Dislocation

In [ ]:
hua_id = "63ff8e895367c2261b4cb2ef"

value_loss = "60354810e379f22e16560dbd"
bg_data = "603545f2dcda03378087e708"
result_name = f"galveston-pop-disl-results_combined_damage"
seed = 1111
pop_dis = PopulationDislocation(client)

pop_dis.set_input_dataset("building_dmg", combined_dmg_ike)
pop_dis.set_parameter("result_name", result_name)
pop_dis.set_parameter("seed", seed)

pop_dis.load_remote_input_dataset("block_group_data", bg_data)
pop_dis.load_remote_input_dataset("value_loss_param", value_loss)
pop_dis.load_remote_input_dataset("housing_unit_allocation", hua_id)

pop_dis.run_analysis()

population_dislocation_result = pop_dis.get_output_dataset("result")
dislocation = population_dislocation_result.get_dataframe_from_csv(low_memory=False)

## Buyout Model Analyses

In [ ]:
fema_buyout_cap = 321291.600
residential_archetypes = [1, 2, 3, 4, 5, 17]

hua_id = "63ff8e895367c2261b4cb2ef"

buyout_decision = BuyoutDecision(client)
buyout_decision.set_parameter("fema_buyout_cap", fema_buyout_cap)
buyout_decision.set_parameter("residential_archetypes", residential_archetypes)
buyout_decision.set_parameter("result_name", "galveston_buyout")

buyout_decision.load_remote_input_dataset("buildings", bldg_dataset_id)
buyout_decision.load_remote_input_dataset("housing_unit_allocation", hua_id)
buyout_decision.set_input_dataset("past_building_damage", combined_dmg_ike)
buyout_decision.set_input_dataset("future_building_damage", combined_dmg_ike_100yrs)
buyout_decision.set_input_dataset("population_dislocation", population_dislocation_result)

buyout_decision.run_analysis()
buyout_result = buyout_decision.get_output_dataset("result")
buyout_df = buyout_result.get_dataframe_from_csv()

#Convert pandas dataframe to geopandas dataframe
buyout_df['geometry'] = buyout_df['geometry'].apply(loads)
buyout_df = gpd.GeoDataFrame(buyout_df, geometry='geometry')

## Results

### 1) Tenure status

In [ ]:
buyout_df['ownership'] = buyout_df['ownership'].replace({1: 'Owner', 2: 'Renter'})

owners = (buyout_df['ownership'] == 'Owner').sum()
renters = (buyout_df['ownership'] == 'Renter').sum()

normal_offer_owners = round(buyout_df.loc[buyout_df['ownership'] == 'Owner', 'housing_unit_appraisal_value'].sum())
normal_offer_renters = round(buyout_df.loc[buyout_df['ownership'] == 'Renter', 'housing_unit_appraisal_value'].sum())

benefit_owner = buyout_df.loc[(buyout_df['dislocated'] == True) & (buyout_df['ownership'] == 'Owner'), 'dislocated'].sum()
benefit_renter = buyout_df.loc[(buyout_df['dislocated'] == True) & (buyout_df['ownership'] == 'Renter'), 'dislocated'].sum()


ownership_summary = {
    '': ['# candidate housing units', 'Total purchase offer', 'Potential benefits (# people not dislocated by a simulated 100-year event)',
         'Potential challenges', 'Potential consequences', 'Equity considerations'],
    'Owners': [owners, f"${normal_offer_owners:,.2f}", benefit_owner,
                '(1) Program participation depends on financial standing, place-based attachments, risk perception, flood exposure, family composition, and community ties. <br/>'+
                '(2) Homeowners are less likely to participate if they have a mortgage. <br/>'+
                '(3) Elderly homeowners (with paid off mortgages) may oppose relocation, due to  place-based attachments and their unwillingness to take out a second mortgage for a more expensive home. <br/>'+
                '(4) Homeowners tend to decline offers if their property has undergone improvements, as home improvements are not factored into the market value. <br/>'+
                '(5) Lower chance of accepting an offer if cost sharing is required. <br/>'+
                '(6) Lower chance of accepting an offer if their house has undergone repair and reconstruction. <br/>'+
                '(7) Owners of substantially damaged properties may have limited post-disaster rebuilding options. While eminent domain is prohibited in buyout programs, the owners  are legally restricted from rebuilding unless they flood-proof the structure or relocate. To avoid such additional expenses, buyout program participation may increase. <br/>'+
                '(8) Homeowners are less inclined to participate if the purchase offer is lower than their expected property value. <br/>'+
                '(9) Homeowners with inadequate property documentation are less likely to participate.', 
                '(1) Possibility of unaffordable housing options upon relocation <br/>'+
                '(2) Losing social network if relocated outside their community', 
                '(1) Sufficient financial incentives can alleviate post-buyout financial burdens and encourage program participation. <br/>'+
                '(2) Up to $22,500 compensation under URA for additional payments such as comparable homes, closing costs, or increased interest costs. <br/>'+
                '(3) Basement coverage through FEMA’s NFIP is limited, and compensation may be needed for contents such as washer, dryer, TV, or food freezer. <br/>'+
                '(4) Consider funding for mental health support of household members who experienced flooding. <br/>'+
                '(5) Fostering relationships with residents and community organizations is vital to establish trust and make buyouts more favorable. <br/>'+
                '(6) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. <br/>'+
                '(7) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation. <br/>'+
                '(8) To counter tax base losses from buyouts, program administrators can incentivize owners to relocate within the county but outside floodplains. <br/>'+
                '(9) If a participating household owes a mortgage, FEMA may pay residual funds after they pay the lienholder. <br/>'+
                '(10) Effective risk communication does not always lead to protective actions like participating in a buyout program. Homeowners responses depend on assessing benefits and their ability to act. Thus, sharing information on buyout benefits, resources like temporary housing options, and available compensations increases confidence in homeowners ability to afford and undergo buyouts.'],    
    'Renters': [renters, f"${normal_offer_renters:,.2f}", benefit_renter,
                '(1) Renters-occupied homes are less likely to undergo buyouts. <br/>'+
                '(2) Foreign national renters are ineligible for federal relocation benefits and post-buyout assistance, intensifying challenges for local governments in providing alternative solutions.',
                '(1) Mandatory relocation <br/>'+
                '(2) Financial burden <br/>'+
                '(3) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability <br/>'+
                '(4) Losing social networks if relocated outside of the community <br/>'+
                '(5) Lower ability to find affordable homes when relocated',  
                '(1) Renters are less likely to carry flood insurance. Local governments may explore strategies to enhance coverage and overall disaster preparedness. <br/>'+
                '(2) Under the URA of 1970, displaced tenants from a property (occupied for 90+ days) can receive a payment (up to $5,250) for renting another property for up to 42 months. <br/>'+
                '(3) Consider funding for mental health support of household members who experienced flooding. <br/>'+
                '(4) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. <br/>'+
                '(5) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation.']
}


ownership_table = pd.DataFrame(ownership_summary, index=None)


ownership_table = ownership_table.style.set_properties(
    subset=ownership_table.columns[1:],  
    **{'text-align': 'left'}
).set_properties(
    subset=ownership_table.columns[0], 
    **{'font-weight': 'bold', 'text-align': 'left'}
).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])
ownership_table

In [ ]:
buyout_df.crs = "EPSG:4326"
buyout_df = buyout_df.to_crs(epsg=3857)

In [ ]:
color_dict = {
    'Owner': 'blue',
    'Renter': 'red'
}

# Create a ListedColormap with your specific colors
colors = [color_dict[ownership] for ownership in buyout_df['ownership'].unique()]
cmap = ListedColormap(colors)

buyout_df.explore(
        column="ownership", # column for archetype info
        tooltip="ownership", 
        #geometry='geometry',
        popup=True,
        tiles='OpenStreetMap',
        cmap=cmap,  # use "Set1" matplotlib colormap
        legend_kwds = {"caption": "Ownership"},
        marker_kwds= {'radius': 4}
)

### 2) Owner-occupied homes broken down by owners' race

In [ ]:
white = buyout_df.loc[(buyout_df['race'] == 1) & (buyout_df['hispan'] == 0) & (buyout_df['ownership'] == 'Owner'), 'huid'].count()
racial_minority = buyout_df.loc[(buyout_df['race'] >= 2) & (buyout_df['hispan'] == 0) & (buyout_df['ownership'] == 'Owner'), 'huid'].count()
hisp = buyout_df.loc[(buyout_df['hispan'] == 1) & (buyout_df['ownership'] == 'Owner'), 'huid'].count()

offer_owner_white = round(buyout_df.loc[(buyout_df['race'] == 1) & (buyout_df['hispan'] == 0) & (buyout_df['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())
offer_owner_racial_minority = round(buyout_df.loc[(buyout_df['race'] >= 2) & (buyout_df['hispan'] == 0) & (buyout_df['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())
offer_owner_hispan = round(buyout_df.loc[(buyout_df['hispan'] == 1) & (buyout_df['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())

benefit_owner_white = buyout_df.loc[(buyout_df['dislocated'] == True) & (buyout_df['ownership'] == 'Owner') & (buyout_df['race'] == 1) & (buyout_df['hispan'] == 0), 'dislocated'].sum()
benefit_owner_racial_minority = buyout_df.loc[(buyout_df['dislocated'] == True) & (buyout_df['ownership'] == 'Owner') & (buyout_df['race'] >= 2) & (buyout_df['hispan'] == 0), 'dislocated'].sum()
benefit_owner_hispan = buyout_df.loc[(buyout_df['dislocated'] == True) & (buyout_df['ownership'] == 'Owner') & (buyout_df['hispan'] == 1), 'dislocated'].sum()

# NOTE: Equity considerations are similar for all homeowners
tenure_race_summary = {
    '': ['# candidate housing units', 'Total purchase offer', 'Potential benefits (# people not dislocated by a simulated 100-year event)',
         'Potential challenges','Potential consequences', 'Equity considerations'],
    'White homeowners': [white, f"${offer_owner_white:,.2f}", benefit_owner_white,
                '(1) Less likely to participate due to higher flood risk tolerance.',
                '(1) If agree to participate, may prefer relocating to majority-White communities.', 
                '(1) Sufficient financial incentives can alleviate post-buyout financial burdens and encourage program participation. <br/>'+
                '(2) Up to $22,500 compensation under URA for additional payments such as comparable homes, closing costs, or increased interest costs. <br/>'+
                '(3) Basement coverage through FEMA’s NFIP is limited, and compensation may be needed for contents such as washer, dryer, TV, or food freezer. <br/>'+
                '(4) Consider funding for mental health support of household members who experienced flooding. <br/>'+
                '(5) Fostering relationships with residents and community organizations is vital to establish trust and make buyouts more favorable. <br/>'+
                '(6) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. <br/>'+
                '(7) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation. <br/>'+
                '(8) To counter tax base losses from buyouts, program administrators can incentivize owners to relocate within the county but outside floodplains. <br/>'+
                '(9) If a participating household owes a mortgage, FEMA may pay residual funds after they pay the lienholder. <br/>'+
                '(10) Effective risk communication does not always lead to protective actions like participating in a buyout program. Homeowners responses depend on assessing benefits and their ability to act. Thus, sharing information on buyout benefits, resources like temporary housing options, and available compensations increases confidence in homeowners ability to afford and undergo buyouts.'],    
    'Racial minority homeowners': [racial_minority, f"${offer_owner_racial_minority:,.2f}", benefit_owner_racial_minority,
                '(1) More prone to relocation due to higher exposure to disasters <br/>'+
                '(2) Reluctance to join the program due to emotional attachment and distrust in government-led programs <br/>'+
                '(3) Properties in Black neighborhoods are valued lower compared to White neighborhoods, posing challenges for finding comparable homes post-buyout.',
                '(1) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability. <br/>'+
                '(2) Losing social network if relocated outside their community',
                ' '],
    'Hispanic homeowners': [hisp, f"${offer_owner_hispan:,.2f}", benefit_owner_hispan,
                '(1) More prone to relocation due to higher exposure to disasters <br/>'+
                '(2) Properties in Hispanic neighborhoods are valued lower compared to White neighborhoods, posing challenges for finding comparable homes post-buyout <br/>'+
                '(3) Reluctance to join the program due to emotional attachment and distrust in government-led programs',
                '(1) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability <br/>'+
                '(2) Losing social network if relocated outside their community',
                ' '],
}

tenure_race_table = pd.DataFrame(tenure_race_summary, index=None)

tenure_race_table = tenure_race_table.style.set_properties(
    subset=tenure_race_table.columns[1:],  
    **{'text-align': 'left'}
).set_properties(
    subset=tenure_race_table.columns[0],  
    **{'font-weight': 'bold', 'text-align': 'left'}
).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])

tenure_race_table

In [ ]:
def categorize_race_tenure(row):
    if row['race'] == 1 and row['hispan'] == 0 and row['ownership'] == 'Owner':
        return 'White alone homeowners'
    elif row['hispan'] == 1 and row['ownership'] == 'Owner':
        return 'Hispanic homeowners'
    elif row['race'] >= 2 and row['hispan'] == 0 and row['ownership'] == 'Owner':
        return 'Racial minority homeowners'
    else:
        return 'Other'


# Apply the function to each row
buyout_df['race_category'] = buyout_df.apply(categorize_race_tenure, axis=1)

colors = ['red','green','blue']
cmap = ListedColormap(colors)

buyout_df[buyout_df['race_category'] != 'Other'].explore(
        column="race_category", # column for archetype info
        tooltip="race_category", 
        #geometry='geometry',
        popup=True,
        tiles='OpenStreetMap',
        cmap=cmap,  # use "Set1" matplotlib colormap
        legend_kwds = {"caption": "Race Category"},
        marker_kwds= {'radius': 4}
)

### 3) Owner-occupied homes broken down by owners' income level

In [ ]:
low_middle_income = buyout_df.loc[(buyout_df['hhinc'].isin([1,2,3])) & (buyout_df['ownership'] == 'Owner'), 'huid'].count()
high_income = buyout_df.loc[(buyout_df['hhinc'].isin([4,5])) & (buyout_df['ownership'] == 'Owner'), 'huid'].count()

offer_owner_low_middle_income = round(buyout_df.loc[(buyout_df['hhinc'].isin([1,2,3])) & (buyout_df['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())
offer_owner_high_income = round(buyout_df.loc[(buyout_df['hhinc'].isin([4,5])) & (buyout_df['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())

benefit_owner_low_middle_income = buyout_df.loc[(buyout_df['hhinc'].isin([1,2,3])) & (buyout_df['ownership'] == 'Owner'), 'dislocated'].sum()
benefit_owner_high_income = buyout_df.loc[(buyout_df['hhinc'].isin([4,5])) & (buyout_df['ownership'] == 'Owner'), 'dislocated'].sum()

# NOTE: Equity considerations are similar for all homeowners
income_tenure_summary = {
    '': ['# candidate housing units', 'Total purchase offer', 'Potential benefits (# people not dislocated by a simulated 100-year event)',
         'Potential challenges','Potential consequences', 'Equity considerations'],
    'Low and middle income homeowners': [low_middle_income, f"${offer_owner_low_middle_income:,.2f}", benefit_owner_low_middle_income,
                '(1) Higher chance of being targeted for program participation as low-value homes sustain more damage <br/>'+
                '(2) Lower chance of accepting an offer if cost sharing is required <br/>'+
                '(3) May accept a low offer due to little power to negotiate the terms <br/>'+
                '(4) Reluctance to relocate due to limited perceived options, lower awareness, and affordability concerns <br/>' +
                '(5) More prone to relocation due to higher exposure to disasters',
                '(1) Less likely to find affordable comparable homes when relocated <br/>'+
                '(2) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability <br/>'+
                '(3) Losing social networks if relocated outside of the community <br/>'+
                '(4) Though voluntary, program participation may resemble forced relocation due to new post-disaster policies mandating costly mitigation measures or increased insurance premiums that are unaffordable to households.',
                '(1) Sufficient financial incentives can alleviate post-buyout financial burdens and encourage program participation. <br/>'+
                '(2) Up to $22,500 compensation under URA for additional payments such as comparable homes, closing costs, or increased interest costs. <br/>'+
                '(3) Basement coverage through FEMA’s NFIP is limited, and compensation may be needed for contents such as washer, dryer, TV, or food freezer. <br/>'+
                '(4) Consider funding for mental health support of household members who experienced flooding. <br/>'+
                '(5) Fostering relationships with residents and community organizations is vital to establish trust and make buyouts more favorable. <br/>'+
                '(6) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. <br/>'+
                '(7) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation. <br/>'+
                '(8) To counter tax base losses from buyouts, program administrators can incentivize owners to relocate within the county but outside floodplains. <br/>'+
                '(9) If a participating household owes a mortgage, FEMA may pay residual funds after they pay the lienholder. <br/>'+
                '(10) Effective risk communication does not always lead to protective actions like participating in a buyout program. Homeowners responses depend on assessing benefits and their ability to act. Thus, sharing information on buyout benefits, resources like temporary housing options, and available compensations increases confidence in homeowners ability to afford and undergo buyouts.'],    
    'High income homeowners': [high_income, f"${offer_owner_high_income:,.2f}", benefit_owner_high_income,
                '(1) Higher chance of ineligibility due to owning high-value properties and not meeting the BCA requirements <br/>'+
                '(2) May not receive a buyout offer as municipalities are concerned about the appearance of federal funds assisting wealthy households <br/>'+
                '(3) May decline the offer due to more financial independence and self-efficacy to implement alternative mitigation measures <br/>'+
                '(4) May decline offers if their property has undergone improvements, as home improvements are not factored into the market value',
                '(1) May need to adopt alternative mitigation measures, if not participating',
                ' ']
}

income_tenure_table = pd.DataFrame(income_tenure_summary, index=None)

income_tenure_table = income_tenure_table.style.set_properties(
    subset=income_tenure_table.columns[1:],  
    **{'text-align': 'left'}
).set_properties(
    subset=income_tenure_table.columns[0],  
    **{'font-weight': 'bold', 'text-align': 'left'}
).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])

income_tenure_table

In [ ]:
def categorize_income(row):
    if row['hhinc'] in [1, 2, 3] and row['ownership'] == 'Owner':
        return 'Low and middle income homeowners'
    elif row['hhinc'] in [4, 5] and row['ownership'] == 'Owner':
        return 'High income homeowners'
    else:
        return 'Other'

# Apply the function to each row
buyout_df['income_category'] = buyout_df.apply(categorize_income, axis=1)
colors = ['red','blue']
cmap = ListedColormap(colors)

buyout_df[buyout_df['income_category'] != 'Other'].explore(
        column="income_category", # column for archetype info
        tooltip="income_category", 
        #geometry='geometry',
        popup=True,
        tiles='OpenStreetMap',
        cmap=cmap,
        legend_kwds = {"caption": "Income Group"},
        marker_kwds= {'radius': 4}
)

### 4) Cluster buyout

In [ ]:
# Calculating the distance of closest building to each building in the inventory
# geometry_col = gpd.GeoSeries.from_wkt(bld_invtry['geometry'])
distance = gpd.GeoDataFrame(geometry=bld_invtry['geometry'])

# Setting the CRS to WGS 84
distance = distance.set_crs(epsg=4326)

# Converting the CRS to a projection that uses feet (e.g., UTM)
distance = distance.to_crs(epsg=3857) 
bld_invtry['closest_distance'] = distance.geometry.apply(lambda x: distance.distance(x).nsmallest(2).iloc[1])

In [ ]:
bld_invtry['closest_distance'].describe()

In [ ]:
df_cluster=buyout_df

In [ ]:
# Setting the threshold for identifying clusters, which is 10 times the standard deviation of calcualted distance
cluster_threshold = 12 * np.std(bld_invtry['closest_distance'])

def identify_clusters(coord1, coord2):
    return geodesic(coord1, coord2).feet
df_cluster['cluster'] = ''

group_counter = 1

for i in range(len(df_cluster)):
    if df_cluster.at[i, 'cluster'] == '':
        df_cluster.at[i, 'cluster'] = f'Group_{group_counter}'
        group_counter += 1

        for j in range(i + 1, len(df_cluster)):
            coord1 = (df_cluster.at[i, 'y'], df_cluster.at[i, 'x'])
            coord2 = (df_cluster.at[j, 'y'], df_cluster.at[j, 'x'])
            distance = identify_clusters(coord1, coord2)

            if distance <= cluster_threshold:
                df_cluster.at[j, 'cluster'] = df_cluster.at[i, 'cluster']

pd.set_option('display.float_format', '{:.15f}'.format)

In [ ]:
# keeping only one housing unit per building to avoid getting clusters just because one building has two housing units (drop duplicate)
df_cluster = df_cluster.drop_duplicates(subset='guid', keep='first')

In [ ]:
# Removing properties that are not in a cluster by dropping the values in "cluster" col that are repeated only once
df_cluster = df_cluster[df_cluster.groupby('cluster')['cluster'].transform('count') > 1]
#df_cluster.shape

In [ ]:
unique_clusters = df_cluster['cluster'].unique()
mapping_dict = {group: f'Cluster {str(i + 1).zfill(2)}' for i, group in enumerate(unique_clusters)}


df_cluster['cluster_label'] = df_cluster['cluster'].map(mapping_dict)
df_cluster = gpd.GeoDataFrame(df_cluster, geometry='geometry')

df_cluster.explore(
        column="cluster_label", # column for archetype info
        tooltip="cluster_label", 
        #geometry='geometry',
        popup=True,
        tiles='OpenStreetMap',
        cmap="Paired",  # use "Set1" matplotlib colormap
        legend_kwds = {"caption": "Clusters"},
        marker_kwds= {'radius': 4}
)

In [ ]:
cluster_buyout_summary = {
    'Potential advantages of cluster buyout': [
        '(1) Incentivizes participation if other neighbors are relocating <br/>' +
        '(2) Losing tax base is a known issue of buyout. Recreational areas developed through cluster buyout, such as parks help increase property values and improve physical and mental health <br/>' +
        '(3) Boosts tourism and recreation by creating new spaces <br/>' +
        '(4) Helps restoring ecological values and enhancing ecosystem services such as improved water quality and reduced risks of future hazards <br/>' +
        '(5) Cluster buyouts align with FEMA open space management objectives of FEMA. While certain communities have repurposed checkerboard-patterned lots for recreation, these lots generally limit open space development opportunities.'
    ]
}

cluster_buyout_table = pd.DataFrame(cluster_buyout_summary, index=None)

cluster_buyout_table = cluster_buyout_table.style.set_properties(
    subset=cluster_buyout_table.columns,  
    **{'text-align': 'left'}
).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])

cluster_buyout_table